In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm

In [2]:
## Unpack Zooniverse Data

In [27]:
df = pd.read_csv("/mnt/machinelearning/BUOW/owl-detector-dataset-2024-03-classifications.csv")
file = '/mnt/machinelearning/BUOW/owl_all.csv'

In [25]:
idx = 0
x = 0
dic = {}
s = set()
df['subject_data'] = df['subject_data'].apply(lambda x:x.replace('null',"None"))
df['subject_data'] = df['subject_data'].astype(str)
df['annotations'] = df['annotations'].astype(str)

for index, row in df.iterrows():
    try:
    
        dic['classification_id'] = row['classification_id']
        dic['user_name'] = row['user_name']
        dic['user_id'] = row['user_id']
        dic['user_ip'] = row['user_ip']
        dic['workflow_id'] = row['workflow_id']
        dic['workflow_name'] = row['workflow_name']
        dic['workflow_version'] = row['workflow_version']
        dic['created_at'] = row['created_at']
        dic['gold_standard'] = row['gold_standard']
        dic['expert'] = row['expert']


        subject_data = eval(row['subject_data'])
        for key in subject_data:
            for key2 in subject_data[key]:
                if key2 != 'retired':
                    dic[key2] = subject_data[key][key2]
                else:
                    for key3 in subject_data[key][key2]:
                        dic[key3] = subject_data[key][key2][key3]

        annotations = eval(row['annotations'])    
        dic['task'] = annotations[0]['task']
        dic['task_label'] = annotations[0]['task_label']


        for i in range(len(annotations[0]['value'])):
            new_dic = dic
            for key in annotations[0]['value'][i]:
                new_dic[key] = annotations[0]['value'][i][key]

            final_entry = pd.DataFrame([new_dic])
            new_df = pd.concat([new_df, final_entry])

            if idx % 10000 == 0:
                file_exists = os.path.isfile(file)
                new_df.to_csv(file, mode='a', header=not file_exists)
                new_df = new_df[0:0]
            idx += 1

    except Exception as e:
        s.add(e)
        x += 1

here


NotImplementedError: 

In [1]:
##Extract only the data we need

In [ ]:
df3 = pd.read_csv('/mnt/machinelearning/BUOW/owl_all.csv')
df2 = pd.read_csv('/mnt/machinelearning/BUOW/BUOW Mapping.csv')

In [ ]:
dic = {}
for index, row in df2.iterrows():
    dic[row['Relative']] = row['Actual']

In [ ]:
df = pd.DataFrame(columns=('Name', 'Path',
                               'X', 'Y', 'Width',
                               'height', 'Label'))
i = 0
for index, row in tqdm(df3.iterrows()):
    data = {'Name': [row['#image' + str(int(row['frame']) + 1)]],
           'Path': [dic[row['#current_dir']] + '/' + row['#image' + str(int(row['frame']) + 1)]], 
           'X': [row['x']], 'Y': [row['y']], 'Width': [row['width']], 'Height': [row['height']],
           'Label': [row['tool_label']]}
    
    df = pd.concat([df, pd.DataFrame(data)]).reset_index(drop=True)
    i += 1
    if i % 10000 == 0:
        df.to_csv('owl_compressed.csv', mode='a', header=False)
        df = df[0:0]
    

In [ ]:
## Get final bounding boxes

In [ ]:
'''
Funtion that return Intersection over Union for 2 bounding boxes

'''
def iou(box1, box2):
    x1, y1, w1, h1, l1, a1 = box1
    x2, y2, w2, h2, l2, a1 = box2
   
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
   
    inter_width = max(0, xi2 - xi1)
    inter_height = max(0, yi2 - yi1)
    inter_area = inter_width * inter_height
   
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area
   
    return inter_area / union_area

In [ ]:
'''
Function that return all unique bounding boxes in an image
'''
def match_boxes(annotations):
    matched_groups = []
    annotations = sorted(annotations, key=lambda row: row[-1])
    for i in annotations:
        if len(matched_groups) == 0:
            matched_groups.append(i)
        else:
            flag = True
            for j in matched_groups:
                if iou(i,j) != 0:
                    flag = False
                    break
            if flag:
                matched_groups.append(i)
    return matched_groups

In [ ]:
# Loading the csv containing annotations
df = pd.read_csv('owl_compressed.csv')

# (Optional) Only keeping the labels we need
df = df[df['Label'].isin(['Adult Owl', 'Juvenile Owl'])]

# Group all annotations by their image
grouped = df.groupby('Path')

In [ ]:
d = pd.DataFrame(columns = ['Name', 'Path', 'X', 'Y', 'Width', 'Height', 'Label'])
count = 0
idx = 0
for name, group in grouped:
    try:
        group['Area'] = group['Width'] * group['Height']
        annotations = group[['X', 'Y', 'Width', 'Height', 'Label', 'Area']].values.tolist()
        matched_groups = match_boxes(annotations)

        if len(matched_groups) != 0:
            for entry in matched_groups:
                data = {'Name': [name.split('/')[-1]], 'Path': [name], 'X': [entry[0]], 'Y': [entry[1]], 
                        'Width': [entry[2]], 'Height': [entry[3]], 'Label': [entry[4]]}
                d = pd.concat([d, pd.DataFrame(data)]).reset_index(drop=True)

                count += 1
                if count % 10000 == 0:
                    print(count)
                    d.to_csv('owl_boxed.csv', mode='a', header=False)
                    d = d[0:0]
    except:
        idx += 1
d.to_csv('owl_boxed.csv', mode='a', header=False)

In [ ]:
'''
Plot a few random images to visualize accuracy of bounding boxes

Note: This method assumes the coordinates are not normalized and represent x_min, y_min, width, height. If you
      have coordinates in other system, please convert them before plotting
'''

num_of_images_to_plot = 50

idx = 0
for index, row in df_new.iterrows():
    
    # Load the image
    img = mpimg.imread(row['Path'])

    # Create a figure and axis
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(img)

    # Create a Rectangle patch
    rect = patches.Rectangle((row['X'], row['Y']), row['Width'], row['Height'], linewidth=2, edgecolor='y', facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)

    # Show the plot with the bounding box
    plt.show()
    
    if idx == num_of_images_to_plot:
        break
    idx += 1

In [ ]:
## Get splits for train and Validate

In [ ]:
df = pd.read_csv('owl_boxed.csv')

In [ ]:
df_adult = df[df['Label'] == 'Adult Owl']
df_juv = df[df['Label'] == 'Juvenile Owl']

In [ ]:
df_adult = df_adult['Path']
df_adult.drop_duplicates()

df_juv = df_juv['Path']
df_juv.drop_duplicates()

In [ ]:
train_adult, test_adult = train_test_split(df_adult, test_size=0.2)
train_juv, test_juv = train_test_split(df_juv, test_size=0.2)

In [ ]:
train = list(train_adult['Path']) + list(train_juv['Path'])
test = list(test_adult['Path']) + list(test_juv['Path'])

In [ ]:
'''
Note:

There might be some common entries between train and test, to avoid bias,
remove the commons, based on the data distribution, you can decide how many
images to keep in train and test, since the splits are random, there's no fixed
method for it
'''

In [ ]:
# Copy images and make their labels 

In [ ]:
df = pd.read_csv('owl_boxed.csv')

In [ ]:
image_folder = '/mnt/machinelearning/BUOW/train/images/'
label_folder = '/mnt/machinelearning/BUOW/train/labels/'

In [ ]:
for i in tqdm(train):
    file_name = '_'.join(i.split('/'))
    shutil.copyfile(i,image_folder+file_name[1:])

In [ ]:
for i in tqdm(train):
    
    d = df[df['Path'] == i]
    file_name = '_'.join(i.split('/'))[1:]
    file_name = (file_name[:-3]) + "txt"
    file_name = label_folder + file_name
    for index, row in d.iterrows():
        with Image.open(row['Path']) as img:
            width, height = img.size
            x_n = (row['X'] + row['Width'] / 2)/ width
            y_n = (row['Y'] + row['Height'] / 2) / height
            w_n = row['Width'] / width
            h_n = row['Height'] / height
            
            if row['Label'] == 'Adult Owl':
                st = str(0) + " " + str(x_n) + " " + str(y_n) + " " + str(w_n) + " " + str(h_n)
            else:
                st = str(1) + " " + str(x_n) + " " + str(y_n) + " " + str(w_n) + " " + str(h_n)
            
            with open(file_name, 'a') as t:
                t.write(st + '\n')

In [ ]:
image_folder = '/mnt/machinelearning/BUOW/valid/images/'
label_folder = '/mnt/machinelearning/BUOW/valid/labels/'

In [ ]:
for i in tqdm(test):
    file_name = '_'.join(i.split('/'))
    shutil.copyfile(i,image_folder+file_name[1:])

In [ ]:
for i in tqdm(test):
    
    d = df[df['Path'] == i]
    file_name = '_'.join(i.split('/'))[1:]
    file_name = (file_name[:-3]) + "txt"
    file_name = label_folder + file_name
    for index, row in d.iterrows():
        with Image.open(row['Path']) as img:
            width, height = img.size
            x_n = (row['X'] + row['Width'] / 2)/ width
            y_n = (row['Y'] + row['Height'] / 2) / height
            w_n = row['Width'] / width
            h_n = row['Height'] / height
            
            if row['Label'] == 'Adult Owl':
                st = str(0) + " " + str(x_n) + " " + str(y_n) + " " + str(w_n) + " " + str(h_n)
            else:
                st = str(1) + " " + str(x_n) + " " + str(y_n) + " " + str(w_n) + " " + str(h_n)
            
            with open(file_name, 'a') as t:
                t.write(st + '\n')

In [ ]:
##Train

In [ ]:
import wandb
from wandb.integration.ultralytics import add_wandb_callback
from ultralytics import YOLO

In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
add_wandb_callback(model, enable_model_checkpointing=True)

In [ ]:
model.train(project='BUOW', data='/mnt/machinelearning/owl.yml', epochs=10, imgsz=1280)

In [ ]:
## Test

In [ ]:
model = YOLO("/home/ctl/animl-py/src/animl/BUOW/train15/weights/best.pt")

In [ ]:
img_path = 'mnt_newnas_buow_CamTrapOriginals_CamTraps--BUOW_2019_MAP_RanchoJamul_Cage 4_R59_E_20190815-0822_R59b_100RECNX_RCNX0367.JPG'

In [ ]:
FILE = "/mnt/machinelearning/BUOW/valid/images/" + img_path

In [ ]:
results = model.predict(source=FILE, conf=0.25, save=True, hide_labels=True)